In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('K11_XF_vip.csv',  encoding = 'utf-8')
df1= pd.read_csv('K11_zone.csv',  encoding = 'utf-8')
df2= pd.read_csv('k11_city.csv',  encoding = 'utf-8')
df3= pd.read_csv('k11_country.csv',  encoding = 'utf-8')
df4= pd.read_csv('k11_province.csv',  encoding = 'utf-8')
df5=pd.read_csv('k11_GENTRYEVENT.csv', encoding = 'utf-8')
df6=pd.read_csv('k11_kdp_bindvipinfo.csv', encoding = 'utf-8')
df7=pd.read_csv('k11_vipgraderule.csv',  encoding = 'utf-8')
# read csv from 


/Users/ansonsuen/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,3,4,5,6,7,8,9,26,27,28,33,48,70,96,97,99,101,102,103,108,111,112,113,114,115,117,120,122,127,128,129,130,132,148,153,154,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,179,190,197,198,200,203,210,211,213,215,217,220,222,224,229,230,231,232,256,264,266,275,276,277,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,308,319,322,333,334,335,351,373,374,377,379,383,387,388,389,390,397) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
def clean_email(df):
    # Apply 'NaN ' if Matches  @.com 
    df['XF_VIPEMAIL']=df['XF_VIPEMAIL'].str.replace(r'(.*)\@.com\b(.*)', 'NaN',regex=True).replace('..uk','.uk').replace(np.nan,'NaN').replace('.con','.com')
    df['XF_VIPEMAIL']= np.where(df['XF_VIPEMAIL'].str.endswith('.con|.co？m|.cpm|.comb|.commq|.conm|.ocm|.cokm|..comt|.comh|？¤com|.lom|como|.cdom|.vom|.xom|.copm|qqcom|.comn|.comg'), df['XF_VIPEMAIL'].str.replace('.con','.com'),df['XF_VIPEMAIL'])
    #if HK02 or HK04 has NaN changed to default@k11.com"
    df['XF_VIPEMAIL']=np.where(df.XF_VIPEMAIL.str.contains(r'NaN') 
                               & (df['XF_ISSUESTORE'].str.contains(r'HK02')|df['XF_ISSUESTORE'].str.contains(r'HK04')),df.XF_VIPEMAIL.replace('NaN',"default@k11.com"),  df['XF_VIPEMAIL'] )
    #if HK03 has NaN changed to @k11.com"
    df['XF_VIPEMAIL']=np.where(df.XF_VIPEMAIL.str.contains(r'NaN') 
                               & (df['XF_ISSUESTORE'].str.contains(r'HK03')),df.XF_VIPEMAIL.replace('NaN',"@K11.com"),  df['XF_VIPEMAIL'] )
    #Apply 'default@k11.com' if not contain 'default@k11.com' when it is HK02  and HK04
    df['XF_VIPEMAIL'] = np.where(~df.XF_VIPEMAIL.str.contains(r'@')& (df['XF_ISSUESTORE'].str.contains(r'HK02')|df['XF_ISSUESTORE'].str.contains(r'HK04')),'default@k11.com', df['XF_VIPEMAIL'])
    #Apply 'default@k11.com' if not contain '@K11.com' when it is HK02  and HK04
    df['XF_VIPEMAIL'] = np.where(~df.XF_VIPEMAIL.str.contains(r'@')& (df['XF_ISSUESTORE'].str.contains(r'HK03')),'@K11.com', df['XF_VIPEMAIL'])
    
    return df

In [3]:
def xf_remarks1(df):
    df['XF_REMARKS1_COPY'] = df['XF_REMARKS1'].str.replace('[', ',')
    df['XF_REMARKS1_COPY'] = df['XF_REMARKS1_COPY'].str.replace(']', '')
    df['XF_REMARKS1_COPY'] = np.where(~df['XF_ISSUESTORE'].str.contains(r'HK01'),df.XF_REMARKS1_COPY.str.split(pat='TOP|/,BU|/ ,NO'),  ',' )# splitting to ready for multiple columns 
    df['Other']=df['XF_REMARKS1_COPY'].str[0]# getting the 1st element
    df['Other']=df['Other'].str.replace('/ ,','')
    df['TOP_3_CAT']=df['XF_REMARKS1_COPY'].str[1]
    df['TOP_3_CAT']=df['TOP_3_CAT'].str.replace('/ ,','')
    df['TOP_3_CAT']=df['TOP_3_CAT'].str.replace('3 CAT','TOP 3 CAT')
    df['TOP_3_BD']=df['XF_REMARKS1_COPY'].str[2]
    df['TOP_3_BD']=df['TOP_3_BD'].str.replace('3 BD','TOP 3 BD')
    df['BU_MBR']=df['XF_REMARKS1_COPY'].str[3]
    df['BU_MBR']=df['BU_MBR'].str.replace('MBR','BU MBR')
    df['NO._of_property']=df['XF_REMARKS1_COPY'].str[4]
    df['NO._of_property']=df['NO._of_property'].str.replace('. OF PROPERTY IN NWD','NO. OF PROPERTY IN NWD')
    return df

In [4]:
def merge(df,df1,df2,df3,df4,df5,df6,df7):
    df=df.merge(df1,left_on='XF_ZONE', right_on='ZONECODE', how='left').merge(df1,left_on='XF_WORK_ZONE', right_on='ZONECODE', how='left').merge(df2,left_on='XF_CITY', right_on='CITYCODE', how='left').merge(df2,left_on='XF_WORK_CITY', right_on='CITYCODE', how='left').merge(df3,left_on='XF_WORK_COUNTRY', right_on='COUNTRYCODE', how='left').merge(df3,left_on='XF_COUNTRY', right_on='COUNTRYCODE', how='left').merge(df4,left_on='XF_PROVINCE', right_on='PROVINCECODE', how='left').merge(df4,left_on='XF_WORK_PROVINCE', right_on='PROVINCECODE', how='left').merge(df5,left_on='XF_VIPCODE', right_on='VIPCODE', how='left').merge(df6,left_on='XF_VIPCODE', right_on='K11_VIPCODE', how='left').merge(df7,left_on='XF_GRADE', right_on='XF_GRADE', how='left').drop(columns=['XF_ZONE','XF_CITY','XF_COUNTRY','XF_PROVINCE', 'K11_VIPCODE','XF_REMARKS1_COPY','XF_REMARKS2_COPY','XF_GRADE','VIPCODE','ZONECODE_x','ZONECODE_y','CITYCODE_x','CITYCODE_y','COUNTRYCODE_x','COUNTRYCODE_y','PROVINCECODE_x','PROVINCECODE_y','XF_WORK_COUNTRY','XF_WORK_PROVINCE','XF_WORK_CITY','XF_WORK_ZONE','XF_REMARKS1','XF_REMARKS2']).rename(columns={'ZONE_x':'XF_ZONE','ZONE_y':'XF_WORK_ZONE',
                     'CITY_x':'XF_CITY','CITY_y':'XF_WORK_CITY','COUNTRY_x':'XF_WORK_COUNTRY','COUNTRY_y':'XF_COUNTRY',
                    'PROVINCE_x':'XF_PROVINCE','PROVINCE_y':'XF_WORK_PROVINCE'}) # left join and rename fields 
    return df



In [5]:
def xf_remarks2(df):
    df['XF_REMARKS2_COPY']=df['XF_REMARKS2'].str.replace(';',',')
    df['XF_REMARKS2_COPY'] = np.where(~df['XF_ISSUESTORE'].str.contains(r'HK041'),df.XF_REMARKS2.str.split(r"\bTop\b|\bMembership\b|\bNumber\b"),  ',' )
    df['XF_REMARKS2_Top_3_Categories']=df['XF_REMARKS2_COPY'].str[1]# getting the 1st element
    df['XF_REMARKS2_Top_3_Categories']=df['XF_REMARKS2_Top_3_Categories'].str.replace('3 Categories','K11 Top 3 Categories').str.replace(r'[K11\s]*$','')
    df['XF_REMARKS2_Top_3_Brand']=df['XF_REMARKS2_COPY'].str[2]
    df['XF_REMARKS2_Top_3_Brand']=df['XF_REMARKS2_Top_3_Brand'].str.replace('3 Brand',' K11 Top 3 Brand').str.replace(r'[BU\s]*$','')
    df['XF_REMARKS2_BU_Membership']=df['XF_REMARKS2_COPY'].str[3]
    df['XF_REMARKS2_BU_Membership']=df['XF_REMARKS2_BU_Membership'].str.replace(':','BU Membership:')
    df['XF_REMARKS2_Number_of_Property_in_NWD']=df['XF_REMARKS2_COPY'].str[4]
    df['XF_REMARKS2_Number_of_Property_in_NWD']=df['XF_REMARKS2_Number_of_Property_in_NWD'].str.replace('of Property','Number of Property in NWD')
    return df

In [6]:
import re
def main(df):
    # change to str type 
    df[['XF_REMARKS1', 'XF_REMARKS2', 'XF_SURNAME','XF_GIVENNAME','XF_TELEPHONE','XF_VIPEMAIL','XF_ENGFAMILYNAME','XF_CHNFAILYNAME']] = df[['XF_REMARKS1', 'XF_REMARKS2', 'XF_SURNAME','XF_GIVENNAME','XF_TELEPHONE','XF_VIPEMAIL','XF_ENGFAMILYNAME','XF_CHNFAILYNAME']].astype(str)
    df[['XF_REMARKS1', 'XF_REMARKS2', 'XF_SURNAME','XF_GIVENNAME','XF_TELEPHONE','XF_VIPEMAIL','XF_ENGFAMILYNAME','XF_CHNFAILYNAME']]=df[['XF_REMARKS1', 'XF_REMARKS2', 'XF_SURNAME','XF_GIVENNAME','XF_TELEPHONE','XF_VIPEMAIL','XF_ENGFAMILYNAME','XF_CHNFAILYNAME']].replace(np.nan,'NaN').replace('nan','NaN')
    
    
    df.XF_TELEPHONE=df.XF_TELEPHONE.str.replace('.0','').str.replace('00000000','')
    #Apply 00000000 when telephone has contain letters 
    df['XF_TELEPHONE'] = np.where(df.XF_TELEPHONE.str.contains(r'[a-zA-Z]') ,'00000000' , df['XF_TELEPHONE'])
    
  
    
    #Apply . when XF_CHNFAILYNAME is 'NaN' and is not hk01
    df['XF_CHNFAILYNAME']=np.where(df.XF_CHNFAILYNAME.str.contains(r'NaN')
                                   &(~df['XF_ISSUESTORE'].str.contains(r'HK01')) ,df.XF_CHNFAILYNAME.replace('NaN',"."),  df['XF_CHNFAILYNAME'] )
    #Apply '.' when XF_ENGFAMILYNAME is 'NaN'  and is not hk01      
    df['XF_ENGFAMILYNAME']=np.where(df.XF_ENGFAMILYNAME.str.contains(r'NaN') 
                                    &(~df['XF_ISSUESTORE'].str.contains(r'HK01')) ,df.XF_ENGFAMILYNAME.replace('NaN',"."),  df['XF_ENGFAMILYNAME'] )    
   #Apply'.' when XF_ENGFAMILYNAME is '.'  and is not hk01      
    df['XF_ENGFAMILYNAME']=np.where(((df.XF_ENGFAMILYNAME.str.contains(r'^([^\w\s])'))
                                     |df.XF_ENGFAMILYNAME.str.contains(r'^([\d+\+])')
                                     |df.XF_SURNAME.str.contains(r'NaN')) 
                                     &(df['XF_ISSUESTORE'].str.contains(r'HK01')) , '.',df['XF_ENGFAMILYNAME'] )
    # Apply '.' when contain non word character("\W") and all white spaces (/s) or decimal and not HK01
    df['XF_SURNAME']=np.where(((df.XF_SURNAME.str.contains(r'^([^\W\s])'))
                               |df.XF_SURNAME.str.contains(r'^([\d+\+])')
                               |df.XF_SURNAME.str.contains(r'NaN')
                               |df.XF_SURNAME.str.contains(r'N/A'))
                               &(~df['XF_ISSUESTORE'].str.contains(r'HK01')),'.' , df['XF_SURNAME'] )
    # Apply '.' when not contain alphabetical, number and not HK01
    df.XF_SURNAME=np.where(~df.XF_SURNAME.str.contains(r'([a-zA-Z])')
                           &~df.XF_SURNAME.str.contains(r'^([\u4e00-\u9fff])')
                           &(~df.XF_ISSUESTORE.str.contains('HK01')),'.',df.XF_SURNAME)
    # Apply '.' when contain non word character("\W") and all white spaces (/s) or decimal and not HK01       
    df['XF_GIVENNAME']=np.where(((df.XF_GIVENNAME.str.contains(r'^([^\W\s])'))
                                 |df.XF_GIVENNAME.str.contains(r'^([\d+\+])')) 
                                 &(~df['XF_ISSUESTORE'].str.contains(r'HK01')) ,'.' ,  df['XF_GIVENNAME'] )
     # Apply '.' when contain non word character("\W") and all white spaces (/s) or decimal and not HK01
    df['XF_CHNFAILYNAME']=np.where(((df.XF_CHNFAILYNAME.str.contains(r'^([^\W\s])'))
                                    |df.XF_CHNFAILYNAME.str.contains(r'^([\d+\+])')) 
                                    &(~df['XF_ISSUESTORE'].str.contains(r'HK01')) ,'.' , df['XF_CHNFAILYNAME'] )
    xf_remarks2(df)
    xf_remarks1(df)
    clean_email(df)

    # HK01 cleansing 
    # Apply '.' when contain non word character("\W") and all white spaces (/s) or decimal and HK01
    df['XF_ENGFAMILYNAME']=np.where(((df.XF_ENGFAMILYNAME.str.contains(r'^([^\W\s])'))
                                     |df.XF_ENGFAMILYNAME.str.contains(r'^([\d+\+])')) 
                                     &df['XF_ISSUESTORE'].str.contains(r'HK01') ,'N/A' , df['XF_ENGFAMILYNAME'] )
    # Apply '.' when contain non word character("\W") and all white spaces (/s) or decimal and HK01
    df['XF_SURNAME']=np.where(((df.XF_SURNAME.str.contains(r'^([^\W\s])'))
                               |df.XF_SURNAME.str.contains(r'^([\d+\+])')) 
                               &df['XF_ISSUESTORE'].str.contains(r'HK01') , 'N/A' , df['XF_SURNAME'] )
    # Apply '.' when contain non word character("\W") and all white spaces (/s) or decimal and HK01
    df['XF_GIVENNAME']=np.where(((df.XF_GIVENNAME.str.contains(r'^([^\W\s])'))
                                 |df.XF_GIVENNAME.str.contains(r'^([\d+\+])')) 
                                 &df['XF_ISSUESTORE'].str.contains(r'HK01') , 'N/A' , df['XF_GIVENNAME'] )
    # Apply '.' when contain non word character("\W") and all white spaces (/s) or decimal and HK01
    df['XF_CHNFAILYNAME']=np.where(((df.XF_CHNFAILYNAME.str.contains(r'^([^\W\s])'))
                                    |df.XF_CHNFAILYNAME.str.contains(r'^([\d+\+])')) 
                                    &df['XF_ISSUESTORE'].str.contains(r'HK01') , 'N/A' , df['XF_CHNFAILYNAME'] )
    
    # Apply np.nan when contain 'NaN'
    df.loc[((df['XF_GIVENNAME'] == 'NaN') 
            |(df['XF_CHNFAILYNAME'] == 'NaN') 
            |(df['XF_ENGFAMILYNAME'] == 'NaN') 
            |(df['XF_REMARKS1'] == 'NaN') 
            |(df['XF_REMARKS2'] == 'NaN') 
            |(df['XF_SURNAME'] == 'NaN'))
            &(df['XF_ISSUESTORE']=='HK01') , ['XF_REMARKS1', 'XF_REMARKS2', 'XF_SURNAME','XF_GIVENNAME','XF_ENGFAMILYNAME','XF_CHNFAILYNAME']] = np.nan
    


    # convert data type(to date)
    df['XF_ISSUEDATE']=pd.to_datetime(df['XF_ISSUEDATE']).dt.date
    df['XF_JOINTDATE']=pd.to_datetime(df['XF_JOINTDATE']).dt.date
    df['XF_LASTPURDATE']=pd.to_datetime(df['XF_LASTPURDATE'],errors = 'coerce').dt.date
    df['XF_EXPIRYDATE']=pd.to_datetime(df['XF_EXPIRYDATE'],errors = 'coerce').dt.date
    df['XF_LASTRENEWDATE']=pd.to_datetime(df['XF_LASTRENEWDATE'],errors = 'coerce').dt.date
    df['XF_GENTRYCLUB_EXPIRYDATE']=pd.to_datetime(df['XF_GENTRYCLUB_EXPIRYDATE'],errors = 'coerce').dt.date
    df['XF_ATELIER_EXPIRYDATE']=pd.to_datetime(df['XF_ATELIER_EXPIRYDATE'],errors = 'coerce').dt.date
    df['XF_ISSUEDATE']=pd.to_datetime(df['XF_ISSUEDATE'], errors = 'coerce')
    df=merge(df,df1,df2,df3,df4,df5,df6,df7)
    
    return df
    

In [7]:
df8=main(df)



<ipython-input-6-5419632b89cf>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.XF_TELEPHONE=df.XF_TELEPHONE.str.replace('.0','').str.replace('00000000','')
/Users/ansonsuen/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-5-1f3307f153c7>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['XF_REMARKS2_Top_3_Categories']=df['XF_REMARKS2_Top_3_Categories'].str.replace('3 Categories','K11 Top 3 Categories').str.replace(r'[K11\s]*$','')
<ipython-input-5-1f3307f153c7>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['XF_REMARKS2_Top_3_Brand']=df['XF_REMARKS2_Top_3_Brand'].str.replace('3 Brand',' K11 Top 3 Brand').str.replace(r'[BU\s]*$','')
<ipython-input-3-e934662da9f6>:2

In [8]:
pd.set_option('display.max_columns', None)

#df8.XF_AGEGROUP= df8.XF_AGEGROUP.apply(lambda )
df8.iloc[1:20]

,XF_VIPCODE,XF_SURNAME,XF_GIVENNAME,XF_ADDRESS1,XF_ADDRESS2,XF_ADDRESS3,XF_ADDRESS4,XF_TELEPHONE,XF_TELEPHONE2,XF_VIPID,XF_DOB,XF_BIRTHDAYYYYY,XF_BIRTHDAYMM,XF_BIRTHDAYDD,XF_SEX,XF_JOINTDATE,XF_EXPIRYDATE,XF_ISSUEDATE,XF_ISSUESTORE,XF_ISSUETILLID,XF_FIRSTPURDATE,XF_LASTPURDATE,XF_ACCUMPUR,XF_TTLPURCOUNT,XF_ACTIVE,XF_REMARKS1,XF_REMARKS2,XF_POSTAL,XF_VIPEMAIL,XF_DEFSALESMAN,XF_LASTBONUSRESETDATE,XF_GROUPVALUETIME,XF_GROUPID0,XF_GROUPID1,XF_GROUPID2,XF_GROUPID3,XF_GROUPID4,XF_GROUPID5,XF_GROUPID6,XF_GROUPID7,XF_GROUPID8,XF_GROUPID9,XF_GROUPID10,XF_GROUPID11,XF_GROUPID12,XF_GROUPID13,XF_GROUPID14,XF_GROUPID15,XF_GROUPID16,XF_GROUPID17,XF_GROUPID18,XF_GROUPID19,XF_DLFLAG,XF_LASTMODBY,XF_LASTMODTIME,XF_LASTUPLOADTIME,XF_ISSUEBY,XF_ISSUETIME,XF_LASTRENEWDATE,XF_LASTRENEWSTORECODE,XF_NEWACCUMPUR,XF_NEWTTLPURCOUNT,XF_BANKCARDNO,XF_CURRBONUSUSED,XF_PREVBONUSUSED,XF_MOBILENOTE,XF_JOURNAL,XF_ISMAINVIP,XF_VIPCARDTYPE,XF_VIPCARDNO,XF_VIPACCOUNTNO,XF_CLUB,XF_JOINCLUBDATE,XF_INFOSOURCE,XF_APPLICATIONID,XF_APPLICATIONDATE,XF_CERTIFICATETYPE,XF_PASSWORDFLAG,XF_PASSWORD,XF_REGION,XF_EDUCATIONCODE,XF_MARITALSTATUS,XF_INCOMECODE,XF_INDUSTRYCODE,XF_SMSCONTACT,XF_EMAILCONTACT,XF_PHONECONTACT,XF_POSTCONTACT,XF_CURRENTBONUS,XF_LASTUPDOWNDATE,XF_HADCHILD,XF_AGEOFCHILD,XF_LASTEXPIRYDATE,XF_INTEREST,XF_HAVESENDPROMPTMAIL,XF_LANGUAGE,XF_LIKEPRODUCT,XF_TOCHINATIMES,XF_TOCHINAREASON,XF_SALESMEMOPHOTO,XF_CONTINENT,XF_TITLE,XF_ENGFAMILYNAME,XF_CHNFAILYNAME,XF_NAMEONCARD,XF_MONTHLYINCOME,XF_COMPANYNAME,XF_BUSINESSTITLE,XF_OCCUPATION,XF_TELEPHONE3,XF_VISITK11REASON,XF_ALLOWSENDPROMOMSG,XF_CATOFPROMOMSG,XF_ALLOWSENDANOTHERPROMOMSG,XF_ALLOWSHAREMYINFO,XF_COMPOFSHAREMYINFO,XF_AGEGROUP,XF_REFERENCES,XF_CHILDAGEGROUP,XF_TERMINALIP,XF_ISAUTOUPGRADE,XF_OPENCARDREDEEMBONUS,XF_REQUESTED,XF_LIKEPRODUCTOTHER,XF_INTERESTOTHER,XF_VISITK11REASONOTHER,XF_TOCHINATIMESOTHER,XF_TOCHINAREASONOTHER,XF_FATHERCODE,XF_FATHERTYPE,XF_REMAINUNCHANGED,XF_MODIFYORIGIAL,XF_FRIENDCIRCLECODE,XF_REFERENCESNAME,XF_REFERENCESTELEPHONE,XF_STORECODE,XF_EMPLOYEENO,XF_BELONGTO,XF_OPENCARDCONDITION,XF_AVERAGECONSUMER,XF_ADDRESS0,XF_ISAGREESTATEMENT,XF_BLACKCARDMODIFIER,XF_ATTENTIONMEMBER,XF_ISARTSCLUB,XF_INFORMAL_MEMBER,XF_JOBNATURE,XF_CHANNEL,XF_INTERESTACTIVITY,XF_INTERESTACTIVITYOTHER,XF_TRADESOURCES,XF_AGE_BYSYS,XF_JOIN_ARTIST_GRADE,XF_FAMILY_CHN_NAME_1,XF_FAMILY_ENG_NAME_1,XF_FAMILY_SEX_1,XF_FAMILY_CHN_NAME_2,XF_FAMILY_ENG_NAME_2,XF_FAMILY_SEX_2,XF_FAMILY_CHN_NAME_3,XF_FAMILY_ENG_NAME_3,XF_FAMILY_SEX_3,XF_FAMILY_CHN_NAME_4,XF_FAMILY_ENG_NAME_4,XF_FAMILY_SEX_4,XF_FAMILY_CATEGORY_1,XF_FAMILY_CATEGORY_2,XF_FAMILY_CATEGORY_3,XF_FAMILY_CATEGORY_4,XF_SHOWNAMEONCARDTYPE,XF_LASTCLEARBONUSDATE,XF_CURRENTREWARDBONUS,XF_REFERENCECARDNO,XF_CURRENTREWARDBONUS_CS,XF_CHILDRENNUMBER,XF_SCHOOL,XF_CLASS,XF_KIDSCOURSES,XF_ISPARKLANDLORD,XF_ADULTSVIPCODE,XF_ADULTSNAMEONCARD,XF_MAIL_SEND_FLAG,XF_INTEACTIVITIES,XF_ISNEWWORDVIP,XF_ISNEWWORDVIPOTHERS,XF_STAFFCODE,XF_UPGRADE_GIFT_FLAG,XF_UPGRADE_SEQNO,XF_REQUESTED_SOURCE,XF_REQUESTED_RELATIONSHIP,XF_CHANGTIME,XF_FACEBOOK_ID,XF_WECHAT_ID,XF_GOOGLE_ID,XF_TELEPHONECOUNTRYCODE,XF_HEADPHOTO,XF_ISPHONEAUTHENTICATION,XF_CONFIRMREGISTEREMAIL,XF_SPOUSENAME,XF_NATIONALITY,XF_WECHAT,XF_PERSONALASSISTANT,XF_COSTMAINBRAND,XF_SPENDMONTHNUM,XF_CHILDRENNAMES,XF_FOODRESTRICTIONS,XF_PETSNAME,XF_TOPSIZE,XF_TOPSIEZOTHER,XF_BOTTOMSIZE,XF_BOTTOMSIZEOTHER,XF_SHOESIZE,XF_SHOESIZEOTHER,XF_WEARINGPIRECE,XF_WEARINGPIRECEOTHER,XF_LOVEFOODS,XF_LOVEFOODSOTHER,XF_LOVEDRINKS,XF_LOVEDRINKSOTHER,XF_LOVECOLOUR,XF_LOVECOLOUROTHER,XF_BUSINESSCARDPHOTO,XF_UNIONID,XF_ISWEDDINGCARD,XF_WEDDINGCARDEXPIRYDATE,XF_WEDDINGACTIVATIONDATE,XF_SARECOMMEND,XF_MIZUHO_STAFFCODE,XF_CHILDSEX1,XF_CHILDSEX2,XF_CHILDSEX3,XF_CHILDAGE1,XF_CHILDAGE2,XF_CHILDAGE3,XF_GRADUATIONLEVEL,XF_ISDRIVER,XF_WORKINTST,XF_CARPARKUSER,XF_PARTNERSHIPTYPE,XF_PARTNERSHIPCODE,XF_TELEPHONECOUNTRYCODE2,XF_TELEPHONECOUNTRYCODE3,XF_CHILDBIRTHDDAYYYYY_1,XF_CHILDBIRTHDDAYYYYY_2,XF_CHILDBIRTHDDAYYYYY_3,XF_CHILDBIRTHDDAYMM_1,XF_CHILDBIRTHDDAYMM_2,XF_CHIL

In [33]:
pd.set_option('display.max_rows', None)
df.XF_SURNAME[df['XF_ISSUESTORE']=='HK03'].value_counts()

.           4849
婉鳴             1
志伟             1
兆堂             1
詩雯             1
國治             1
WING           1
景順             1
西華             1
景軒             1
巧蓮             1
詠茵             1
J              1
光祖             1
sunenti        1
yee hong       1
Name: XF_SURNAME, dtype: int64

In [27]:
def cleans_names(df):
    regExp = '、|^&+-%*/=!>';
    df['XF_GIVENNAME']=df['XF_GIVENNAME'].str.replace(regExp,'')
    df['XF_ENGFAMILYNAME_valid'] = np.where(df.XF_ENGFAMILYNAME.str.contains(r'^([\u4e00-\u9fff].{1}$)') & df['XF_ISSUESTORE'].str.contains(r'HK04'), df['XF_GIVENNAME'], df['XF_ENGFAMILYNAME'])
    return df
    

    
    